In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from sklearn.linear_model import LinearRegression
import re

In [ ]:
def extract_number(text, default=10):
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())
    return default


In [ ]:
#Streamlit ayarları / Page settings
st.set_page_config(page_title="Nüfus Chatbotu", layout="centered")
st.title("Dünya Nüfusu Chatbot Analiz Aracı")
st.markdown( """ Bu uygulama, dünya nüfus verileri ile etkileşimli şekilde çalışmanızı sağlar.  
Soru sorarak grafik, tablo, istatistik ve makine öğrenmesi tahminleri alabilirsiniz.  
**Örnekler:**  
- `En kalabalık 10 ülke`  
- `Hindistan'ın 2025 nüfusu nedir?`  
- `Türkiye için tahmin et`  
- `En hızlı büyüyen 15 ülkeyi göster`
- `Germanay ve Turkey karşılaştır`
""")

In [ ]:
#CSV dosyasını oku /  Read the CSV file
df= pd.read_csv("WorldPopulationByCountry.csv")

In [ ]:
#Sütun adlarını düzenle / Rename columns for easier use
df.rename(columns={
    "pop2025": "Population_2025",
    "pop2050": "Population_2050",
    "growthRate": "Growth_Rate",
    "worldPercentage" : "World_Percentage",
    " density" : "Density"
}, inplace=True)

In [ ]:
#Ml MODELİ Lineer Regression
model=LinearRegression()
model.fit(df[["Population_2025"]] , df["Population_2050"])

In [ ]:
def chatbot_reply(user_input):
    user_input = user_input.lower()

    # Çoklu ülke karşılaştırması kontrolü / Check if multiple countries are mentioned
    countries_in_text = [country for country in df["country"] if country.lower() in user_input]
    num_countries = len(countries_in_text)

    if num_countries == 2:
        country1, country2 = countries_in_text
        row1 = df[df["country"].str.lower() == country1.lower()]
        row2 = df[df["country"].str.lower() == country2.lower()]

        pop25_1, pop50_1 = row1["Population_2025"].values[0], row1["Population_2050"].values[0]
        pop25_2, pop50_2 = row2["Population_2025"].values[0], row2["Population_2050"].values[0]

        st.subheader(f"📊 {country1} ve {country2} Karşılaştırması / Comparison")
        comp_df = pd.DataFrame({
            "Ülke / Country": [country1, country2],
            "2025 Nüfusu": [pop25_1, pop25_2],
            "2050 Nüfusu": [pop50_1, pop50_2],
            "Büyüme Oranı": [row1["Growth_Rate"].values[0], row2["Growth_Rate"].values[0]]
        })

        st.dataframe(comp_df)

        fig, ax = plt.subplots(figsize=(6, 4))
        sns.barplot(x=["2025-"+country1, "2050-"+country1, "2025-"+country2, "2050-"+country2],
                    y=[pop25_1, pop50_1, pop25_2, pop50_2], ax=ax)
        ax.set_title(f"{country1} vs {country2} Nüfus Değişimi / Population Change")
        st.pyplot(fig)
        return

    elif num_countries == 1:
        country = countries_in_text[0]
        row = df[df["country"].str.lower() == country.lower()]
        pop25 = int(row["Population_2025"].values[0])
        pop50 = int(row["Population_2050"].values[0])
        rate = float(row["Growth_Rate"].values[0])

        st.subheader(f"📍 {country} Verileri / Data")
        st.write(f"👶 2025: {pop25:,}")
        st.write(f"👴 2050: {pop50:,}")
        st.write(f"📈 Büyüme Oranı: {rate:.2%}")

        fig, ax = plt.subplots(figsize=(5, 4))
        sns.barplot(x=["2025", "2050"], y=[pop25, pop50], ax=ax)
        ax.set_title(f"{country} Nüfus Değişimi")
        st.pyplot(fig)
        return

    # Diğer sorgular: en kalabalık, hızlı büyüyen vs
    if "en kalabalık" in user_input or "most populated" in user_input:
        n = extract_number(user_input, default=10)
        top = df.sort_values(by="Population_2050", ascending=False).head(n)

        st.subheader(f"📊 2050'de En Kalabalık {n} Ülke")
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(data=top, x="Population_2050", y="country", ax=ax)
        st.pyplot(fig)
        st.dataframe(top[["country", "Population_2050"]])
        return

    elif "en hızlı" in user_input or "fastest" in user_input:
        n = extract_number(user_input, default=10)
        top = df.sort_values(by="Growth_Rate", ascending=False).head(n)

        st.subheader(f"📈 En Hızlı Büyüyen {n} Ülke")
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(data=top, x="Growth_Rate", y="country", ax=ax)
        st.pyplot(fig)
        st.dataframe(top[["country", "Growth_Rate"]])
        return

    elif "tahmin" in user_input or "predict" in user_input:
        for country in df["country"]:
            if country.lower() in user_input:
                pop25 = df[df["country"].str.lower() == country.lower()]["Population_2025"].values[0]
                predicted = model.predict(np.array([[pop25]]))[0]
                st.success(f"{country} için model tahmini 2050 nüfusu: {int(predicted):,}")
                return
        return st.warning("Ülke bulunamadı / Country not found.")

    else:
        return st.info("❓ Soru tanınmadı. Örnekler:\n- Türkiye verilerini getir\n- Türkiye ve Almanya'yı karşılaştır\n- En kalabalık 7 ülke")


In [ ]:
# Kullanıcının soru ve yanıt geçmişini sakla
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Kullanıcıdan soru al
user_input = st.text_input("📝 Soru Girin / Ask a Question")

# Soru geldiyse işleme al
if user_input:
    st.session_state.chat_history.append(("👤 Siz", user_input))
    
    # Yanıt üret
    with st.spinner("Yanıtlanıyor..."):
        # chatbot_reply fonksiyonundan çıktı alma
        response = chatbot_reply(user_input)
    
    # Eğer chatbot bir şey yazdıysa (örneğin bilgi veya uyarı), onu da sakla
    if response is not None:
        st.session_state.chat_history.append(("🤖 Chatbot", response))

# Sohbet geçmişini göster
st.subheader("📜 Sohbet Geçmişi / Chat History")
for sender, msg in st.session_state.chat_history:
    st.markdown(f"**{sender}:** {msg}")
